In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions


from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from tqdm import tqdm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


бостон - градиент уходит в 0  
ирис - спустя пару итераций в 0

In [3]:
dataset = load_breast_cancer()
# dataset = load_boston()
df = pd.DataFrame(dataset['data'])
target = dataset['target']
df = (df - df.mean())/(df.max() - df.min())
df0 = df.copy()
print(df.shape)
print(target)
df.head()

(569, 30)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.182815,-0.301307,0.213053,0.146813,0.198968,0.531437,0.495081,0.487976,0.306758,0.335139,...,0.324113,-0.222474,0.385173,0.279792,0.196998,0.399079,0.351207,0.518192,0.335156,0.229268
1,0.304923,-0.051392,0.282848,0.284671,-0.104905,-0.078833,-0.004450,0.105621,0.000193,-0.129057,...,0.310239,-0.060427,0.256680,0.264308,-0.056585,-0.065649,-0.024432,0.245339,-0.029717,0.033282
2,0.263274,0.066295,0.262808,0.232497,0.119524,0.170416,0.254453,0.392549,0.129991,-0.059132,...,0.259723,-0.003924,0.225304,0.203602,0.079452,0.165163,0.142341,0.441216,0.140399,0.023837
3,-0.128132,0.036874,-0.099434,-0.114014,0.416536,0.550761,0.357546,0.279726,0.396657,0.729621,...,-0.048353,0.021929,-0.041791,-0.076898,0.511335,0.593799,0.331239,0.491044,0.736693,0.584115
4,0.291671,-0.167388,0.298051,0.272369,0.035567,0.087292,0.255859,0.275253,-0.001323,-0.083564,...,0.223081,-0.240065,0.223810,0.170669,0.033226,-0.047797,0.102086,0.164583,-0.105806,-0.047001


In [4]:
num_nan_cols = 2
nan_fraction = 0.3
nan_cols = np.random.random_integers(0, df.shape[1] - 1, num_nan_cols)
# print(df.isnull().mean())
for col in set(nan_cols):
    df.loc[df.sample(int(nan_fraction * len(df))).index, col] = np.nan
# print(df.isnull().mean())
# print(df.isnull().mean())

/Users/kek/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: This function is deprecated. Please call randint(0, 29 + 1) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
nan_coords = np.array(np.where(df.isnull().values)).T
print('Num nan places: {}'.format(nan_coords.shape[0]))

Num nan places: 340


In [ ]:
df1 = df.loc[:, df.isnull().sum() == 0]
df2 = df.fillna(df.mean())
print(df1.shape, df2.shape)
arr0 = df0.values
arr1 = df1.values
arr2 = df2.values
print(arr0.shape, arr1.shape, arr2.shape)
print(np.isnan(arr0).sum(), np.isnan(arr1).sum(), np.isnan(arr2).sum())

In [ ]:
arr2.shape

# Пилим необходимые функции

df - исходная с прочерками    
df0 - копия исходной таблицы без прочерков  
df1 - сокращенная до ненулевых признаков  
df2 - исходная с восстановленными  
arr0,arr1,arr2 аналогично

In [ ]:
n = len(df0)
# таблица старых значений
Exp1_table = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(n):
        Exp1_table[i][j] = np.exp(-norm(arr1[i] - arr1[j])**2)
        
P1_table = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(n):
        a = Exp1_table[j][i]
        b = Exp1_table[i].sum() - 1 # 1 = Exp1_table[i][i]
        P1_table[j][i] = a / b

In [ ]:
def get_exp2_table():
    Exp2_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(n):
            Exp2_table[i][j] = np.exp(-norm(arr2[i] - arr2[j])**2)
    return Exp2_table

def get_p2_table():
    Exp2_table = get_exp2_table()
    P2_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(n):
            a = Exp2_table[j][i]
            b = Exp2_table[i].sum() - 1 # 1 = Exp1_table[i][i]
            P2_table[j][i] = a / b
    return P2_table
P2_table = get_p2_table()

In [ ]:
def KL():
    s = 0
    for i in range(n):
        for j in range(n):
            s += P1_table[j][i] * np.log(P1_table[j][i] / P2_table[j][i])
    return s

def get_grad(i1, i2):
    def get_i_part(i):
        d = 2*(arr2[i1][i2] - arr2[i][i2])
        s = (P1_table[i1][i]+P1_table[i][i1]) - \
            P2_table[i1][i]*(1+P1_table[i][i]) - \
            P2_table[i][i1]*(1+P1_table[i1][i1])
        return s*d
    return sum(get_i_part(i) for i in range(n) if i!=i1)

def get_full_grad(nan_coords):
    n_gaps = len(nan_coords)
    grad = np.zeros(n_gaps)
    for i in range(n_gaps):
        i1, i2 = nan_coords[i]
        grad[i] = get_grad(i1, i2)
    return grad

Тестовый прогон

# Iterative process

In [ ]:
c = KL()
c

In [ ]:
mae = functions.get_mae(arr0, arr2, nan_coords)
mae

In [ ]:
cs = []
maes = []
msqes = []
accs = []

mae = functions.get_mae(arr0, arr2, nan_coords)
msqe = functions.get_msqe(arr0, arr2, nan_coords)
acc = functions.get_acc(arr2, target)

cs.append(c)
maes.append(mae)
msqes.append(msqe)
accs.append(acc)

In [ ]:
alpha = 0.1
max_iter = 30
print('\tIteration {} //  c {:.6} // mae {:.6} // rmse {:.6} // acc {}'.format(0, c, mae, msqe, acc))
for i in range(max_iter):
    grad = get_full_grad(nan_coords)
    for j, (x,y) in enumerate(nan_coords):
        arr2[x, y] = arr2[x, y] - alpha*grad[j]
    P2_table = get_p2_table()
    c = KL()
    mae = functions.get_mae(arr0, arr2, nan_coords)
    msqe = functions.get_msqe(arr0, arr2, nan_coords)
    acc = functions.get_acc(arr2, target)
    
    accs.append(acc)
    cs.append(c)
    maes.append(mae)
    msqes.append(msqe)
    print('\tIteration {} // c {:.6} // mae {:.6} // rmse {:.6} // acc {}'.format(i+1, c,mae, 
                                                                                                   msqe, acc))